from [Likelihood encoding of categorical features
](https://www.kaggle.com/tnarik/likelihood-encoding-of-categorical-features)

In [1]:
import pandas as pd 
import numpy as np 
import matplotlib.pyplot as plt
%matplotlib inline

In [2]:
from sklearn.model_selection import StratifiedKFold, KFold

In [3]:
df_train = pd.read_pickle('../input/feats/df_trn_feat0.pkl')
df_test  = pd.read_pickle('../input/feats/df_test_feat0.pkl')

In [5]:
#### params ###

# np.random.seed(13)
# target = 'deal_probability'
# feature = 'param_1'
# data = df_train.iloc[:10000,:].copy()

#####
def impact_coding(data, feature, target='deal_probability', seed=0):
    ## most of the code from: https://www.kaggle.com/tnarik/likelihood-encoding-of-categorical-features
    ''' 
    input
    -----
    data   : origin training data(include y 'target')
    feature: categorical name to be encoded 
    target : name of y
    
    output
    -----
    encoded result 
    
    oof mean cv 
    
    default mean : global mean 
    '''
    n_folds = 10
    n_inner_folds = 5
    print('encode:{}\ttarget:{}'.format(feature,target))
    print('n_folds:{},\tn_inner_folds:{}'.format(n_folds,n_inner_folds))
    impact_coded = pd.Series()

    oof_default_mean = data[target].mean() # Gobal mean to use by default

    kf = KFold(n_splits=n_folds, shuffle=True,random_state=seed)

    oof_mean_cv = pd.DataFrame()
    split = 0
    print('encode oof no:',end=' ')
    for oof_no, (infold, oof) in enumerate(kf.split(data[feature])):
        print(oof_no,end='\t')
        impact_coded_cv = pd.Series()
        kf_inner = KFold(n_splits=n_inner_folds, shuffle=True,random_state=seed)
        inner_split = 0
        infold_inner_mean_cv = pd.DataFrame() ## inner_oof_mean_cv <-> infold_inner_mean_cv
        oof_default_inner_mean = data.iloc[infold][target].mean()

        for infold_inner, oof_inner in kf_inner.split(data.iloc[infold]):
            # The mean to apply to the inner oof split   (a 1/n_folds % based on the rest)
            infold_inner_mean = data.iloc[infold_inner].groupby(by=feature)[target].mean()

            ############# 不清楚用途???? ##################
#             impact_coded_cv_tmp = data.iloc[infold].copy()
#             impact_coded_cv_tmp = impact_coded_cv_tmp[feature].map(oof_mean).fillna(oof_default_inner_mean)
#             impact_coded_cv = impact_coded_cv.append(impact_coded_cv_tmp)
            #############################################

            # Also populate mapping (this has all group -> mean for all inner CV folds)
            infold_inner_mean_cv = infold_inner_mean_cv.join(pd.DataFrame(infold_inner_mean), rsuffix=inner_split, how='outer') ## inner_oof_mean_cv <-> infold_inner_mean_cv 
            infold_inner_mean_cv.fillna(value=oof_default_inner_mean, inplace=True) ## inner_oof_mean_cv <-> infold_inner_mean_cv  
            inner_split += 1

        # Also populate mapping
        oof_mean_cv = oof_mean_cv.join(pd.DataFrame(infold_inner_mean_cv), rsuffix=split, how='outer')
        oof_mean_cv.fillna(value=oof_default_mean, inplace=True)
        split += 1

        #### map to oof ##############################
        impact_coded_tmp = data.iloc[oof].copy()
        infold_inner_mean_cv_ = infold_inner_mean_cv.mean(axis=1) ## oof_inner_mean
        impact_coded_tmp = impact_coded_tmp[feature].map(infold_inner_mean_cv_).fillna(oof_default_mean)        
        impact_coded = impact_coded.append(impact_coded_tmp)
        #############################################
    print('')
    print('------'*10)
    return impact_coded.sort_index(), oof_mean_cv.mean(axis=1), oof_default_mean

In [7]:
df_trn_encoded = pd.DataFrame()
df_test_encoded = df_test.copy()

categorical_features = [
    'uidx',
    'iidx',
    'region_city_label',
    'param_1','param_2','param_3',
    'user_type',
    'parent_category_name',
    'category_name',
    'image_top_1'    
]

for feat in categorical_features:    
    encode_mean, oof_mean_cv, default_mean = impact_coding(df_train, feat, seed=0)
    df_trn_encoded['mean_' + feat] = encode_mean.astype('float16')
    df_test_encoded['mean_'+ feat] = df_test_encoded[feat].map(oof_mean_cv).fillna(default_mean).astype('float16')

encoded_names = ['mean_' + e for e in categorical_features]
df_test_encoded = df_test_encoded[encoded_names]
print('complete')

encode:uidx	target:deal_probability
n_folds:10,	n_inner_folds:5
encode oof no: 0	1	2	3	4	5	6	7	8	9	
------------------------------------------------------------
encode:iidx	target:deal_probability
n_folds:10,	n_inner_folds:5
encode oof no: 0	1	2	3	4	5	6	7	8	9	
------------------------------------------------------------
encode:region_city_label	target:deal_probability
n_folds:10,	n_inner_folds:5
encode oof no: 0	1	2	3	4	5	6	7	8	9	
------------------------------------------------------------
encode:param_1	target:deal_probability
n_folds:10,	n_inner_folds:5
encode oof no: 0	1	2	3	4	5	6	7	8	9	
------------------------------------------------------------
encode:param_2	target:deal_probability
n_folds:10,	n_inner_folds:5
encode oof no: 0	1	2	3	4	5	6	7	8	9	
------------------------------------------------------------
encode:param_3	target:deal_probability
n_folds:10,	n_inner_folds:5
encode oof no: 0	1	2	3	4	5	6	7	8	9	
------------------------------------------------------------
encode:user_

In [9]:
df_trn_encoded.dtypes

mean_uidx                    float16
mean_iidx                    float16
mean_region_city_label       float16
mean_param_1                 float16
mean_param_2                 float16
mean_param_3                 float16
mean_user_type               float16
mean_parent_category_name    float16
mean_category_name           float16
mean_image_top_1             float16
dtype: object

In [10]:
df_test_encoded.head()

,mean_uidx,mean_iidx,mean_region_city_label,mean_param_1,mean_param_2,mean_param_3,mean_user_type,mean_parent_category_name,mean_category_name,mean_image_top_1
1503424,0.139160,0.119568,0.136719,0.067749,0.075134,0.094727,0.149536,0.075806,0.060730,0.049438
1503425,0.139160,0.172729,0.109497,0.282715,0.139160,0.139160,0.149536,0.123901,0.248413,0.139160
1503426,0.139160,0.150635,0.122498,0.244995,0.139160,0.139160,0.149536,0.175537,0.171021,0.177368
1503427,0.027832,0.120239,0.137573,0.313232,0.198242,0.139160,0.149536,0.179810,0.252441,0.139160
1503428,0.139160,0.150635,0.187866,0.249878,0.139160,0.139160,0.149536,0.075806,0.198730,0.287842


##### save it 

In [11]:
store = pd.HDFStore('../input/feats/df_mean_target_cv2.h5')
store['df_trn_encoded'] = df_trn_encoded.astype('float16')
store['df_test_encoded'] = df_test_encoded.astype('float16')
store.close()

In [12]:
df_trn_encoded.head()

,mean_uidx,mean_iidx,mean_region_city_label,mean_param_1,mean_param_2,mean_param_3,mean_user_type,mean_parent_category_name,mean_category_name,mean_image_top_1
0,0.130127,0.188232,0.123291,0.090759,0.139160,0.139160,0.149536,0.075806,0.198730,0.083923
1,0.027832,0.145996,0.139038,0.126221,0.139160,0.139160,0.149536,0.179810,0.192261,0.152710
2,0.333008,0.160034,0.125244,0.124268,0.139160,0.139160,0.149536,0.175537,0.171021,0.202026
3,0.670410,0.085815,0.135132,0.334473,0.139160,0.139160,0.124634,0.075806,0.198730,0.337891
4,0.194214,0.178833,0.136719,0.282715,0.364014,0.373047,0.149536,0.262939,0.278076,0.318359


In [10]:
df_trn_encoded.astype('float16').head()

,mean_uidx,mean_iidx,mean_region_city_label,mean_param_1,mean_param_2,mean_param_3,mean_user_type,mean_parent_category_name,mean_category_name,mean_image_top_1
0,0.130127,0.188232,0.123291,0.090759,0.139160,0.139160,0.149536,0.075806,0.198730,0.083923
1,0.027832,0.145996,0.139038,0.126221,0.139160,0.139160,0.149536,0.179810,0.192261,0.152710
2,0.333008,0.160034,0.125244,0.124268,0.139160,0.139160,0.149536,0.175537,0.171021,0.202026
3,0.670410,0.085815,0.135132,0.334473,0.139160,0.139160,0.124634,0.075806,0.198730,0.337891
4,0.194214,0.178833,0.136719,0.282715,0.364014,0.373047,0.149536,0.262939,0.278076,0.318359


____
for test purpose


In [45]:
#### params ###
np.random.seed(13)
target = 'deal_probability'
feature = 'param_1'
data = df_train.iloc[:10000,:].copy()


n_folds = 5
n_inner_folds = 4

impact_coded = pd.Series()

oof_default_mean = data[target].mean() # Gobal mean to use by default (you could further tune this)

kf = KFold(n_splits=n_folds, shuffle=True)

oof_mean_cv = pd.DataFrame()
split = 0

#####

for infold, oof in kf.split(data[feature]):
    impact_coded_cv = pd.Series()
    kf_inner = KFold(n_splits=n_inner_folds, shuffle=True)
    inner_split = 0
    infold_inner_mean_cv = pd.DataFrame() ## inner_oof_mean_cv <-> infold_inner_mean_cv
    oof_default_inner_mean = data.iloc[infold][target].mean()
    
    for infold_inner, oof_inner in kf_inner.split(data.iloc[infold]):
        # The mean to apply to the inner oof split   (a 1/n_folds % based on the rest)
        infold_inner_mean = data.iloc[infold_inner].groupby(by=feature)[target].mean()
        
        ############# 不清楚用途???? ##################
        impact_coded_cv_tmp = data.iloc[infold].copy()
        impact_coded_cv_tmp = impact_coded_cv_tmp[feature].map(oof_mean).fillna(oof_default_inner_mean)
        impact_coded_cv = impact_coded_cv.append(impact_coded_cv_tmp)
        #############################################

        # Also populate mapping (this has all group -> mean for all inner CV folds)
        infold_inner_mean_cv = infold_inner_mean_cv.join(pd.DataFrame(infold_inner_mean), rsuffix=inner_split, how='outer') ## inner_oof_mean_cv <-> infold_inner_mean_cv 
        infold_inner_mean_cv.fillna(value=oof_default_inner_mean, inplace=True) ## inner_oof_mean_cv <-> infold_inner_mean_cv  
        inner_split += 1

    # Also populate mapping
    oof_mean_cv = oof_mean_cv.join(pd.DataFrame(infold_inner_mean_cv), rsuffix=split, how='outer')
    oof_mean_cv.fillna(value=oof_default_mean, inplace=True)
    split += 1
    
    ##### 不清楚用途 ###########################################
    impact_coded_tmp = data.iloc[oof].copy()
    infold_inner_mean_cv_ = infold_inner_mean_cv.sum(axis=1)
    impact_coded_tmp = impact_coded_tmp[feature].map(infold_inner_mean_cv_).fillna(oof_default_mean)
    impact_coded = impact_coded.append(impact_coded_tmp)
    #############################################

